In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')

In [2]:
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder\
    .appName('Spark Test App')\
    .master('spark://172.31.82.177:7077')\
    .getOrCreate()

In [5]:
sample_df = spark.read.load(
    'hdfs://172.31.82.177:9000/export.csv',
    format = 'csv',
    inferSchema = 'true',
    sep = ',',
    header = 'true'
)

In [6]:
sample_df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [9]:
sorted_df = sample_df.sort('cca2', 'timestamp')

In [15]:
sorted_df.write.csv(
    'hdfs://172.31.82.177:9000/export_sorted.csv',
    header = 'true',
    mode = 'overwrite'
)

In [21]:
check_sorted_df = spark.read.load(
    'hdfs://172.31.82.177:9000/export_sorted_2.csv',
    format = 'csv',
    inferSchema = 'true',
    sep = ',',
    header = 'true'
)

In [22]:
check_sorted_df.show()

+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|                  cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            5|     1217|  AE| ARE|United Arab Emirates|      501|device-mac-501e4O...|      48|  213.42.16.154|    24.0|yellow|     54.0|Celsius|  16|1458444054343|
|            0|      915|  AR| ARG|           Argentina|      227|meter-gauge-2273p...|      34|  200.71.230.81|   -34.6| green|   -58.38|Celsius|  15|1458444054251|
|            1|     1189|  AR| ARG|           Argentina|      319|meter-gauge-319Y3...|      54| 200.71.236.145|   -34.6|yellow|   -58.38|Celsius|  25|1458444054287|
|   

In [23]:
check_sorted_df.createOrReplaceTempView('sorted_temp')

spark.sql('SELECT COUNT(1) FROM sorted_temp').show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [24]:
spark.stop()